# SELECT basics

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app00")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@1577ad41

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@1d5c79dd

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val world = hiveCxt.table("sqlzoo.world")

world: DataFrame = [name: string, continent: string ... 6 more fields]

## 1. Introducing the `world` table of countries

The example uses a WHERE clause to show the population of "France". Note that strings (pieces of text that are data) should be in "single quotes";

**Modify it to show the population of Germany**

In [5]:
(world.filter($"name" === "Germany")
 .select("population")
 .showHTML())

population
8.31493E7


## 2. Scandinavia

Checking a list The word **IN** allows us to check if an item is in a list. The example shows the name and population for the countries "Brazil", "Russia", "India" and "China".

**Show the name and the population for "Sweden", "Norway" and "Denmark".**

In [6]:
val listVal = Seq("Sweden", "Norway", "Denmark")
(world.filter($"name".isin(listVal: _*))
 .select($"name", $"population")
 .showHTML())

name,population
Denmark,5822763.0
Norway,5367580.0
Sweden,1.0338368E7


listVal: Seq[String] = List("Sweden", "Norway", "Denmark")

## 3. Just the right size

Which countries are not too small and not too big? `BETWEEN` allows range checking (range specified is inclusive of boundary values). The example below shows countries with an area of 250,000-300,000 sq. km. Modify it to show the country and the area for countries with an area between 200,000 and 250,000.

In [7]:
(world.filter($"area".between(200000, 250000))
 .select($"name", $"area")
 .showHTML())

name,area
Belarus,207600.0
Ghana,238533.0
Guinea,245857.0
Guyana,214969.0
Laos,236800.0
Romania,238391.0
Uganda,241550.0
United Kingdom,242900.0


In [8]:
spark.stop()